In [12]:
from PIL import Image
from typing import *
import numpy as np
from numpy.linalg import norm
import cv2

In [13]:
class RedSquareGenerator:
    def __init__(
        self,
        image_size: Tuple[int, int] = (100, 100),
        square_count: int = 17,
        square_size: Tuple[int, int] = (11, 11),
    ):
        self.__image_size: int = image_size
        self.__target_count: int = square_count
        self.__sq_h, self.__sq_w = square_size

    def generate(self) -> Generator[None, None, Image.Image]:
        RED = np.array(  (255, 0, 0), dtype=np.uint8)
        BLUE = np.array(  (0, 0, 255), dtype=np.uint8)
        BLACK = np.array((  0, 0, 0), dtype=np.uint8)
        START_POSITION = (0, 0)

        y_s, x_s = START_POSITION

        for i in range(self.__target_count):
            im = Image.new("RGB", self.__image_size, "white")
            pixels = im.load()

            for y in range(self.__sq_h):
                for x in range(self.__sq_w - 1):
                    pixels[y_s + y, x_s + x + i] = tuple(RED + BLUE)
                pixels[y_s + y, x_s + x + 1 + i] = tuple(np.array(BLACK, dtype=np.uint8))

            yield im

In [14]:
from time import time

rsg = RedSquareGenerator(square_count=10)

# .save("./5.png")
image = next(rsg.generate())
img = np.array(image)

start = time()

purple_thresh = cv2.inRange(img, (250, 0, 250), (255, 0, 255))
black_thresh = cv2.inRange(img, (0, 0, 0), (5, 5, 5))

purple_square_coordinates = np.where(purple_thresh == 255)
black_square_coordinates = np.where(black_thresh == 255)

robot_coords = np.mean(purple_square_coordinates, axis=1)
robot_facing = np.mean(black_square_coordinates, axis=1) - robot_coords
robot_facing /= norm(robot_facing)

end = time()

print(f"time: {end-start}")
print(f"coords: {robot_coords/img.shape[:2]}")
print(f"facing: {robot_facing}")

time: 0.000993967056274414
coords: [0.045 0.05 ]
facing: [1. 0.]


In [15]:
from itertools import combinations

r = [
    np.array([0, 0]),
    np.array([1, 2]),
    np.array([2, 1]),
    np.array([3, 0]),
    np.array([4, 2]),
    ]

c = np.array(list(combinations(r, 2)))
guide_vector_points = np.mean(c[:, 0, :], axis=0), np.mean(c[:, 1, :], axis=0)
guide_vector_points

(array([1. , 0.8]), array([3. , 1.2]))

In [16]:
from sympy.geometry import Line2D, Point2D

line = Line2D(Point2D(guide_vector_points[0]), Point2D(guide_vector_points[1]))

r_last = r[-1]
r_projected = line.projection(r_last)
r_projected

Point2D(107/26, 37/26)

In [36]:
facing = np.array([robot_facing,] * 5)

t = [0, 0.3, 0.5, 0.7, 1]

mean_facing = np.mean(facing, axis=0)

dr = norm(np.diff(r, axis=0), axis=1) # [ |1 2| |1 -1| |1 -1| |1 2| ]
dt = np.diff(t, axis=0) # [ 0.3 0.2 0.2 0.3 ]

v_mean = np.sum(dr) / np.sum(dt) # ~7.30

r_predicted = r_projected + v_mean * np.mean(dt, axis=0) * mean_facing

r_predicted

Point2D(5.94052538532106, 1.42307692307692)

In [3]:
data = [cv2.imread(f"./{i}.png", cv2.IMREAD_COLOR) for i in range(1, 5)]
data

[array([[[ 85, 112, 109],
         [ 85, 112, 109],
         [ 90, 118, 113],
         ...,
         [ 98, 114, 113],
         [ 91, 106, 108],
         [ 91, 106, 108]],
 
        [[ 85, 112, 109],
         [ 85, 112, 109],
         [ 90, 118, 113],
         ...,
         [ 98, 114, 113],
         [ 91, 106, 108],
         [ 91, 106, 108]],
 
        [[ 81, 108, 105],
         [ 81, 108, 105],
         [ 81, 108, 103],
         ...,
         [105, 121, 120],
         [100, 115, 117],
         [100, 115, 117]],
 
        ...,
 
        [[120, 114,  85],
         [120, 114,  85],
         [117, 111,  82],
         ...,
         [104,  97,  78],
         [110, 102,  85],
         [110, 102,  85]],
 
        [[124, 118,  89],
         [124, 118,  89],
         [125, 119,  90],
         ...,
         [113, 106,  88],
         [122, 113,  99],
         [122, 113,  99]],
 
        [[124, 118,  89],
         [124, 118,  89],
         [125, 119,  90],
         ...,
         [113, 106,  88],
  

In [36]:
for i in range(1, 5):
    cv2.imshow(f"{i}", data[i-1]) 
cv2.waitKey(0)

13

In [55]:
purple_range1 = (220,50,170), (250,80,200)
purple_range2 = (210,30,110), (230,50,140)

for i in range(1, 5):
    pr1 = cv2.inRange(data[i-1], *purple_range1)
    pr2 = cv2.inRange(data[i-1], *purple_range2)
    dilate = cv2.dilate(np.bitwise_or(pr1, pr2), kernel=np.ones((7,7)))
    white_dots_coords = np.where(dilate != 0)

    r = tuple(map(int, tuple(np.mean(white_dots_coords, axis=1))))[::-1]

    print(r)

    dilate = cv2.cvtColor(dilate, cv2.COLOR_GRAY2BGR)
    dilate = cv2.circle(dilate, r, 5, (0, 0, 255), thickness=5)

cv2.imshow(f"{i}", dilate)
cv2.waitKey(0)

(638, 42)
(638, 72)
(645, 81)
(617, 117)


13

In [56]:
for i in range(1, 5):
    cv2.destroyWindow(f"{i}")
cv2.destroyAllWindows()
cv2.waitKey(1)

-1